In [1]:
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
import pickle
import gc

In [2]:
# Used across all of the plots
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time())

# Levels would have been relatively normal up until Friday 13 March, so take a year before that date
baselineEnd = datetime.datetime.strptime('2020-03-13T23:59:59Z', '%Y-%m-%dT%H:%M:%SZ')
baselineStart = baselineEnd - pd.Timedelta(days=365) + pd.Timedelta(seconds=1)

print('Baseline data from %s to %s' % (baselineStart, baselineEnd))

Baseline data from 2019-03-15 00:00:00 to 2020-03-13 23:59:59


In [ ]:
windowSize = 30

peopleRequestBase = 'https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/data/json/'
peopleRequestVariables = [
    'Walking North East',
    'Walking North West',
    'Walking South East',
    'Walking North',
    'Walking South',
    'Walking East',
    'Walking West'
]

peopleDataRaw = None

for windowStart in dateutil.rrule.rrule(
  dateutil.rrule.DAILY,
  interval=windowSize,
  dtstart=baselineStart,
  until=baselineEnd
):  
    windowEnd = windowStart + pd.Timedelta(days=windowSize) - pd.Timedelta(seconds=1)
    
    if windowEnd > baselineEnd:
        windowEnd = baselineEnd
        
    peopleRequestIRI = ('%s?variable=%s&starttime=%s&endtime=%s') % (
        peopleRequestBase,
        ','.join(str(x).replace(' ', '%20') for x in peopleRequestVariables),
        windowStart.strftime('%Y%m%d%H%M%S'),
        windowEnd.strftime('%Y%m%d%H%M%S')
    )
    
    print('Requesting pedestrian data for %s' % windowStart.strftime('%b %Y'))

    peopleDataWindow = json.loads(
      urllib
        .request
        .urlopen(peopleRequestIRI)
        .read()
        .decode('utf-8')
    )['sensors']
    
    if peopleDataRaw is None:
        peopleDataRaw = peopleDataWindow
    else:
        for sensor in peopleDataWindow:
            for variable in sensor['data']:
                targetSensor = next(s for s in peopleDataRaw if s['Sensor Name'] == sensor['Sensor Name'])
                
                if variable not in targetSensor['data']:
                    continue
                
                targetVariable = targetSensor['data'][variable]
                
                print('  Adding %u items to %u others for %s on %s' % (
                    len(sensor['data'][variable]),
                    len(targetSensor['data'][variable]),
                    variable,
                    targetSensor['Sensor Name']['0']
                ))
                
                targetVariable.extend(sensor['data'][variable])
                
    peopleDataWindow = None
    gc.collect()

Requesting pedestrian data for Mar 2019


In [ ]:
print('Obtained data from %u sensors.' % len(peopleDataRaw))

In [ ]:
pickle.dump(peopleDataRaw, open('../cache/baseline-pedestrian-flow-api-json.pkl', 'wb'))

peopleDataRaw = None
gc.collect()